In [4]:
!pip install nsepython

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from nsepython import *
import pandas as pd
from datetime import date

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
def equity_history_virgin(symbol,series,start_date,end_date):
    url="https://www.nseindia.com/api/historical/cm/equity?symbol="+symbol+"&series=[%22"+series+"%22]&from="+str(start_date)+"&to="+str(end_date)+""
    payload = nsefetch(url)
    return pd.DataFrame.from_records(payload["data"])

In [44]:
import datetime

def fetch_equity_history_with_change(symbol, days=365, series="EQ"):
    end_date = datetime.datetime.now().strftime("%d-%m-%Y")
    start_date = (datetime.datetime.now() - datetime.timedelta(days=days)).strftime("%d-%m-%Y")
    df = equity_history(symbol, series, start_date, end_date)
    if df is None or df.empty:
        print(f"No data returned for {symbol}")
        return None           
    df["daily_change"] = df["CH_CLOSING_PRICE"].pct_change()
    # Uncomment below lines if you want to filter columns or drop first row
    # df = df[['CH_TIMESTAMP', 'daily_change']]
    # df = df.iloc[1:, :]
    print(df)
    return df



In [45]:
fetch_equity_history_with_change("abcd", days=10, series="EQ")

No data returned for abcd


In [46]:
#stocks = ['BAJFINANCE','JSWSTEEL','AXISBANK','M_M','ETERNAL','TATASTEEL','HINDALCO','TRENT','EICHERMOT','TITAN','HDFCBANK','ADANIENT','NTPC','BAJAJ-AUTO','COALINDIA','APOLLOHOSP','JIOFIN','INFY','ADANIPORTS','SBIN','ULTRACEMCO','TECHM','GRASIM','HINDUNILVR','TCS','ITC','POWERGRID','ICICIBANK','HCLTECH','LT','TATACONSUM','ASIANPAINT','RELIANCE']
stocks = ['BAJFINANCE','JSWSTEEL','AXISBANK','M_M','ETERNAL']

In [47]:
# 

Fetching data for: BAJFINANCE
                          _id   CH_SYMBOL CH_SERIES CH_MARKET_TYPE CH_TIMESTAMP                 TIMESTAMP  CH_TRADE_HIGH_PRICE  CH_TRADE_LOW_PRICE  CH_OPENING_PRICE  CH_CLOSING_PRICE  CH_LAST_TRADED_PRICE  CH_PREVIOUS_CLS_PRICE  CH_TOT_TRADED_QTY  CH_TOT_TRADED_VAL  CH_52WEEK_HIGH_PRICE  CH_52WEEK_LOW_PRICE  CH_TOTAL_TRADES       CH_ISIN                 createdAt                 updatedAt  __v SLBMH_TOT_VAL     VWAP   mTIMESTAMP                                                 CA  daily_change
0    6840341a0ff63e5551abaa27  BAJFINANCE        EQ              N   2025-06-04  2025-06-03T18:30:00.000Z              9039.50             8964.50           8975.00           8987.00               8994.50                8979.00             554493       4.988888e+09               9660.00               6375.7            63399  INE296A01024  2025-06-04T11:55:06.396Z  2025-06-04T11:55:06.396Z    0          None  8997.21  04-Jun-2025                                        

In [ ]:
#linear execution 
#from random import randint
# import pandas as pd
# import datetime

# # Initialize the final table

# total = pd.DataFrame()

# # Define time range

# for symbol in stocks:
#     print(f"Fetching data for: {symbol}")
#     try:
#      total= total+fetch_equity_history_with_change(symbol, days=365, series="EQ")
#     except Exception as err:
#         print(f"Error: Skipping symbol {symbol} due to error: {err}")

# # Final review
# print("Final dataset preview:")
# print(total.head())

# parallel execution

from random import randint
import pandas as pd
import datetime
import concurrent.futures

# Initialize the final table
total = pd.DataFrame()

def fetch_wrapper(symbol):
    print(f"Fetching data for: {symbol}")
    try:
        return fetch_equity_history_with_change(symbol, days=365, series="EQ")
    except Exception as err:
        print(f"Error: Skipping symbol {symbol} due to error: {err}")
        return pd.DataFrame()  # Return empty DataFrame on error

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(fetch_wrapper, stocks))

# Combine all results
total = pd.concat(results, ignore_index=True)

# Final review
print("Final dataset preview:")
print(total.head())

Fetching data for: BAJFINANCE
Fetching data for: JSWSTEEL
Fetching data for: AXISBANK
Fetching data for: M_M
Fetching data for: ETERNAL
                          _id   CH_SYMBOL CH_SERIES CH_MARKET_TYPE CH_TIMESTAMP                 TIMESTAMP  CH_TRADE_HIGH_PRICE  CH_TRADE_LOW_PRICE  CH_OPENING_PRICE  CH_CLOSING_PRICE  CH_LAST_TRADED_PRICE  CH_PREVIOUS_CLS_PRICE  CH_TOT_TRADED_QTY  CH_TOT_TRADED_VAL  CH_52WEEK_HIGH_PRICE  CH_52WEEK_LOW_PRICE  CH_TOTAL_TRADES       CH_ISIN                 createdAt                 updatedAt  __v SLBMH_TOT_VAL     VWAP   mTIMESTAMP                                                 CA  daily_change
0    68403419f8805773fac6e581  BAJFINANCE        EQ              N   2025-06-04  2025-06-03T18:30:00.000Z              9039.50             8964.50           8975.00           8987.00               8994.50                8979.00             554493       4.988888e+09               9660.00               6375.7            63399  INE296A01024  2025-06-04T11:55:05.303Z 

OSError: Cannot save file into a non-existent directory: '../inputdata/trading'

In [52]:
total.to_csv('../../inputdata/trading/stock.csv',index=False)
#total.to_csv('./../inputdata/trading/total_may24_jun25.csv', sep='\t', index=False, header=True)
#payload.to_csv('./../inputdata/trading/total_may24_jun25_v2.csv', sep='\t', index=False, header=True)
#column_list=["CH_SYMBOL","CH_SERIES","CH_TIMESTAMP","TIMESTAMP","CH_TRADE_HIGH_PRICE","CH_TRADE_LOW_PRICE","CH_OPENING_PRICE","CH_CLOSING_PRICE","CH_LAST_TRADED_PRICE","CH_PREVIOUS_CLS_PRICE","CH_TOT_TRADED_QTY","CH_TOT_TRADED_VAL","CH_52WEEK_HIGH_PRICE","CH_52WEEK_LOW_PRICE","CH_TOTAL_TRADES","CH_ISIN","createdAt","updatedAt","SLBMH_TOT_VAL"]

#or use columns arg
#total.to_csv('../../inputdata/trading/total_niftyfifty.csv', columns = column_list,index=False)